<h1> Produce interactive bokeh plots for platinum data </h1>

In [1]:
import sys
home_dir=!echo "$HOME"
sys.path.insert(2,home_dir[0]+'/iPyNb/NERSC_Import') 

from __future__ import division
from mpl_numpy_scipy_import import *
from bokeh_pandas_seaborn_import import *
from bokeh.models import CustomJS
from bokeh.plotting import figure, output_file, show

:0: FutureWarning: IPython widgets are experimental and may change in the future.


<h2>Setup. Display in notebook or write to a custum HTML file <br/> -you can view in notebook with output_notebook() or write to an html file</h2>

In [50]:
###################################################################################
#Use this to display interactive plots here in this notebook:
#output_notebook()

###################################################################################
#files produced so far:

#1 Run03 WS
output_file("WS_2013_platinum.html",title='Run03 WS (Platinum)')
data=sio.loadmat('/project/projectdirs/lux/data/platinum/Platinum_Run03_WS.mat')

#2 CH3T Aug2013
#output_file("CH3T_Aug2013_platinum.html",title='CH3T_Aug2013 (Platinum)')
#data=sio.loadmat('/project/projectdirs/lux/data/platinum/Platinum_CH3T_Aug.mat')

#3 CH3T Dec2013
#output_file("CH3T_Dec2013_platinum.html",title='CH3T_Dec2013 (Platinum)')
#data=sio.loadmat('/project/projectdirs/lux/data/platinum/Platinum_CH3T_Dec.mat')

#4 DD data Oct-Dec 2013
#output_file("DD_2013_platinum.html",title='DD_2013 (Platinum)')
#data=sio.loadmat('/project/projectdirs/lux/data/platinum/Platinum_DD_Run03.mat')

#5 Cs137 data Aug 2013
#output_file("Cs_Aug2013_platinumish.html",title='Cs_Aug2013 (Platinum-ish)')
#data=sio.loadmat('/project/projectdirs/lux/data/platinum/Platinum_Cs137_Aug2013.mat')

In [51]:
df=df_from_mat(data)

#energy scale calibration (Run03, g1=0.117, g2=12.05) (slight shift for DD and Dec CH3T)
g1=0.117
g2=12.05

# convert luxstamp to date_num
lux_epoch=md.datestr2num('Jan 1st 2011 at 00:00')
luxstamp2day=df['luxstamp']/10**8/3600/24
luxdatenum=md.num2date(luxstamp2day+lux_epoch)
df['luxdatenum']=luxdatenum

df['log10S2spikyS1']=log10(df['s2area']/df['spikyS1'])
#df['rawlog10S2spikyS1']=log10(df['s2area_raw']/df['spikyS1_raw'])
df['log10S2S1']=log10(df['s2area']/df['s1area'])
df['log10S2S1_raw']=log10(df['s2area_raw']/df['s1area_raw'])
df['s2radius_corr']=sqrt(df['corrected_x']**2 + df['corrected_y']**2)
df['s2radius2_corr']=df['corrected_x']**2 + df['corrected_y']**2
df['energy_spiky']=1/73*(df['spikyS1']/g1 + df['s2area']/g2)
df['energy']=1/73*(df['s1area']/g1 + df['s2area']/g2)

ds=ColumnDataSource(df)

<h2> Create interactive Time histogram </h2>

In [52]:
tools="pan,wheel_zoom,box_zoom,box_select,poly_select,reset,save,hover,click"
hist, edges = np.histogram(df['luxstamp'], density=False, bins=50,normed=0)

edges2day=edges/10**8/3600/24
edgesdatenum=md.num2date(edges2day+lux_epoch)

hist_per_day=hist/(edges2day[1]-edges2day[0])

source = ColumnDataSource({'x':[], 'y':[], 'width':[], 'height':[]})
jscodex="""
    var data = source.get('data');
    var start = range.get('start');
    var end = range.get('end');
    data['%s'] = [start + (end - start) /2];
    data['%s'] = [(end - start)];
    source.trigger('change');
     
    var d2 = ds.get('data');
    index = d2['index'] 
    luxstamp=d2['luxdatenum']
    new_indices = [] 
    for (i = 0; i <= index.length; i++) {  
        if (luxstamp[i]>=start && luxstamp[i]<=end){
            new_indices.push(i) 
        }  
    } 
    selected = ds.get('selected') 
    selected['1d'].indices = new_indices 
    ds.trigger('change');
    
"""

jscodey="""
    var data = source.get('data');
    var start = range.get('start');
    var end = range.get('end');
    data['%s'] = [start + (end - start) /2];
    data['%s'] = [(end - start)];
    source.trigger('change');
 
    
"""

histo=figure(title='Box-Zoom or Pan Here', tools='box_zoom,wheel_zoom,pan,reset', x_axis_type="datetime", 
             plot_width=500, plot_height=200, x_axis_label='Date',y_axis_label='count/day')
histo2=figure(tools='box_zoom,wheel_zoom,pan,reset',x_axis_type="datetime",x_axis_label='Date',
              plot_width=500, plot_height=200,y_axis_label='count/day')


histo.quad(top=hist_per_day, bottom=0, left=edgesdatenum[:-1], right=edgesdatenum[1:], 
     fill_color="#036564", line_color="#033649",alpha=0.5,\
)
histo2.quad(top=hist_per_day, bottom=0, left=edgesdatenum[:-1], right=edgesdatenum[1:], 
     fill_color="#036564", line_color="#033649",alpha=0.5,\
)

rect = Rect(x='x', y='y', width='width', height='height', fill_alpha=0.1,
            line_color='black', fill_color='magenta')


histo.x_range.callback = CustomJS(
    args=dict(source=source, ds=ds, range=histo.x_range), code=jscodex % ('x', 'width'))
histo.y_range.callback = CustomJS(
    args=dict(source=source, range=histo.y_range), code=jscodey % ('y', 'height'))

histo2.add_glyph(source, rect)

#show(histo)

<h2> Define E histogram </h2>

In [53]:
hist, edges = np.histogram(df['energy'], density=False, bins=50,normed=0)

hist_per_keV=hist/(edges2day[1]-edges2day[0])

source2 = ColumnDataSource({'x':[], 'y':[], 'width':[], 'height':[]})

jscodexE="""
    var data = source.get('data');
    var start = range.get('start');
    var end = range.get('end');
    data['%s'] = [start + (end - start) /2];
    data['%s'] = [(end - start)];
    source.trigger('change');
     
    var d2 = ds.get('data');
    index = d2['index'] 
    energy=d2['energy']
    new_indices = [] 
    for (i = 0; i <= index.length; i++) {  
        if (energy[i]>=start && energy[i]<=end){
            new_indices.push(i) 
        }  
    } 
    selected = ds.get('selected') 
    selected['1d'].indices = new_indices 
    ds.trigger('change');
    
"""

jscodeyE="""
    var data = source.get('data');
    var start = range.get('start');
    var end = range.get('end');
    data['%s'] = [start + (end - start) /2];
    data['%s'] = [(end - start)];
    source.trigger('change');
 
    
"""

Ehisto=figure(title='Box-Zoom or Pan Here', tools='box_zoom,wheel_zoom,pan,reset', 
             plot_width=500, plot_height=200, x_axis_label='Energy (keV)',y_axis_label='count/keV')
Ehisto2=figure(tools='box_zoom,wheel_zoom,pan,reset',x_axis_label='Energy (keV)',
              plot_width=500, plot_height=200,y_axis_label='count/keV')


Ehisto.quad(top=hist_per_keV, bottom=0, left=edges[:-1], right=edges[1:], 
     fill_color="#036564", line_color="#033649",alpha=0.5,\
)
Ehisto2.quad(top=hist_per_keV, bottom=0, left=edges[:-1], right=edges[1:], 
     fill_color="#036564", line_color="#033649",alpha=0.5,\
)

rect = Rect(x='x', y='y', width='width', height='height', fill_alpha=0.1,
            line_color='black', fill_color='red')


Ehisto.x_range.callback = CustomJS(
    args=dict(source=source2, ds=ds, range=Ehisto.x_range), code=jscodexE % ('x', 'width'))
Ehisto.y_range.callback = CustomJS(
    args=dict(source=source2, range=Ehisto.y_range), code=jscodeyE % ('y', 'height'))

Ehisto2.add_glyph(source2, rect)

#show(HBox(Ehisto,Ehisto2))

<h1> Scatter plots uncorrected vs corrected variables, linked with histograms </h1>

In [54]:
tools="pan,wheel_zoom,box_zoom,box_select,poly_select,reset,save,hover,click,lasso_select" #crosshair

p1 = figure(title='',plot_width=600, plot_height=400, tools=tools,x_axis_label = "Energy (keVee)",y_axis_label = "log10(S2/S1)")
p2 = figure(title='',plot_width=600, plot_height=400, tools=tools,x_axis_label = "S1 (phd)",y_axis_label = "log10(S2/S1)")
p3 = figure(title='',plot_width=400, plot_height=400, tools=tools,x_axis_label = "x (cm)",y_axis_label = "y (cm)")
p4 = figure(title='',plot_width=400, plot_height=400, tools=tools,x_axis_label = "s2radius^2 (cm^2)",y_axis_label = "drift (us)")

#setup custum hovertool display
hover1 = p1.select(dict(type=HoverTool))
hover1.tooltips = [("index", "$index"),("(x,y)", "($x, $y)"),("E(keVee)", "@energy"),("LUXstamp","@luxstamp"),("Z(us)","@drift"),("R(cm)","@s2radius_corr")]
hover2 = p2.select(dict(type=HoverTool))
hover2.tooltips = [("index", "$index"),("(x,y)", "($x, $y)"),("E(keVee)", "@energy"),("LUXstamp","@luxstamp"),("Z(us)","@drift"),("R(cm)","@s2radius_corr")]
hover3 = p3.select(dict(type=HoverTool))
hover3.tooltips = [("index", "$index"),("(x,y)", "($x, $y)"),("E(keVee)", "@energy"),("LUXstamp","@luxstamp"),("Z(us)","@drift"),("R(cm)","@s2radius_corr")]
hover4 = p4.select(dict(type=HoverTool))
hover4.tooltips = [("index", "$index"),("(x,y)", "($x, $y)"),("E(keVee)", "@energy"),("LUXstamp","@luxstamp"),("Z(us)","@drift"),("R(cm)","@s2radius_corr")]


p1.scatter('energy', 'log10S2S1', source=ds, size=3, color='blue',legend='Energy',fill_alpha=.1,alpha=1,line_color="blue",line_width=1,name='p1')
p1.scatter('energy_spiky', 'log10S2S1', source=ds, size=3, color='red',legend='Energy_spikyS1',fill_alpha=0.1,alpha=.6,name='p1a')
p2.scatter('s1area_raw', 'log10S2S1_raw', source=ds, size=3, color='grey',legend='raw',fill_alpha=.1,alpha=1,name='p2')
p2.scatter('s1area', 'log10S2S1', source=ds, size=3, color='blue',legend='xyz corr',fill_alpha=0.1,alpha=1,name='p2a')
p2.scatter('spikyS1', 'log10S2spikyS1', source=ds, size=3, color='red',legend='spikyS1',fill_alpha=.1,alpha=.6,name='p2b')

#modify glif if there is too much data
if (size(df['s1area'])>5000):
    al=0.2
    p3.scatter('corrected_x','corrected_y',source=ds,name='p3',alpha=al,)
    p4.scatter('s2radius2_corr','drift',source=ds,name='p4',alpha=al)
    select_glyph = Circle(fill_color='blue',line_color='blue',fill_alpha=0.2, line_alpha=0.2) 
    nonselect_glyph = Circle(fill_alpha=0.03, line_alpha=0.03)
    renderer3 = p3.select(name="p3") 
    renderer3.selection_glyph = select_glyph
    renderer3.nonselection_glyph = nonselect_glyph 
    renderer4 = p4.select(name="p4") 
    renderer4.selection_glyph = select_glyph
    renderer4.nonselection_glyph = nonselect_glyph 
else:
    p3.scatter('corrected_x','corrected_y',source=ds,name='p3')
    p4.scatter('s2radius2_corr','drift',source=ds,name='p4')


invisible_glyph = Circle(fill_alpha=0.0, line_alpha=0.0) 
renderer = p1.select(name="p1") 
renderer.selection_glyph = renderer[0].glyph.clone() 
renderer.nonselection_glyph = invisible_glyph 

callback = CustomJS(args=dict(source=ds,s2=source,range=histo2.y_range), code=""" 
    var last = last.get('value')
    var first = first.get('value')
    
    if (last<first){
        first=last;
        first.get('value')=last;
        first.trigger('change')
    }
    
    var data = source.get('data'); 
    luxstamps=data['luxdatenum'];
    index = data['index'] 
    new_indices = [] 
    for (i = first; i <= last; i++) {  
        new_indices.push(i) 
    } 
    selected = source.get('selected') 
    selected['1d'].indices = new_indices 
    source.trigger('change'); 
    
    //update the source of the time histogram, now called s2
        var start = luxstamps[first];
        var end = luxstamps[last];
        var d2 = s2.get('data');
        d2['x'] = [start + (end - start) /2];
        d2['width'] = [(end - start)];
        
        //set y width based on current y_range
        var start = range.get('start');
        var end = range.get('end');
        d2['y'] = [start + (end - start) /2];
        d2['height'] = [(end - start)*0.9];

        s2.trigger('change');
    
""") 


ds.callback = CustomJS(args=dict(source=source,source2=source2, range=histo2.y_range,Erange=Ehisto2.y_range,Erange_x=Ehisto2.x_range), code="""
        var inds = cb_obj.get('selected')['1d'].indices;
        var d1 = cb_obj.get('data');
        luxstamps=d1['luxdatenum'];
        energies=d1['energy'];
        luxstamp_min=luxstamps[inds[0]];
        luxstamp_max=luxstamps[inds[0]];
        energy_min=energies[inds[0]];
        energy_max=energies[inds[0]];
        
        for (i=0; i<=inds.length;i++){
            luxstamp=luxstamps[inds[i]];
            energy=energies[inds[i]];
            if(luxstamp<luxstamp_min){
                luxstamp_min=luxstamp;
            }
            if(luxstamp>luxstamp_max){
                luxstamp_max=luxstamp;
            }
            if(energy<energy_min){
                energy_min=energy;
            }
            if(energy>energy_max){
                energy_max=energy;
            }
        }
        
        //if only one point is selected
        if (inds.length==1){
            luxstamp_min=luxstamp_min*0.9999;
            luxstamp_max=luxstamp_max*1.0001;
            energy_min=energy_min*0.99;
            energy_max=energy_max*1.01;
        }
        
        //no change if nothing is selected
        if (inds.length==0){
            luxstamp_min=luxstamps[0];
            luxstamp_max=luxstamps[luxstamps.length-1];
            energy_min=Erange_x.get('start');   //Math.min([energies['1d]]);
            energy_max=Erange_x.get('end'); 
        }
        
        //update the luxstamp histogram
        var start = luxstamp_min;
        var end = luxstamp_max;
        
        var data = source.get('data');
        data['x'] = [start + (end - start) /2];
        data['width'] = [(end - start)];
        
        //set y width based on current y_range
        var start = range.get('start');
        var end = range.get('end');
        data['y'] = [start + (end - start) /2];
        data['height'] = [(end - start)*0.9];

        source.trigger('change');
        
       //update the energy histogram
        var start = energy_min;
        var end = energy_max;
        
        var data = source2.get('data');
        data['x'] = [start + (end - start) /2];
        data['width'] = [(end - start)];
        
        //set y width based on current y_range
        var start = Erange.get('start');
        var end = Erange.get('end');
        data['y'] = [start + (end - start) /2];
        data['height'] = [(end - start)*0.9];

        source2.trigger('change');
        
        """)


slider_end = Slider(start=0, end=size(ds.data['luxstamp'])-1, value=size(ds.data['luxstamp']), step=1, title="end_index", callback=callback) 
callback.args["last"] = slider_end
slider_start = Slider(start=0, end=size(ds.data['luxstamp'])-1, value=0, step=1, title="start_index", callback=callback) 
callback.args["first"] = slider_start


#this will write an html file
layout=VBox(hplot(slider_start,slider_end),HBox(histo,histo2),HBox(p2,p1),HBox(p3,p4),HBox(Ehisto,Ehisto2))
show(layout)